In [1]:
import sys
import os

# Get the absolute path of the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Add the project root directory to the Python path
sys.path.append(project_root)

In [2]:
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Define the path to the data directory
data_dir = os.path.join(project_root, 'data')

# Example: Access a specific data file in the data directory
data_file_path = os.path.join(data_dir, 'a2rData_train_ED.csv')
df = pd.read_csv(data_file_path)

In [3]:
from AutoML.analyzer import Analyzer
from pprint import pprint

config = Analyzer.dry_run(df)

pprint(config)

Config file not found. Creating custom...
Used a heuristic to define categorical and continuous columns. Please review!


Feature Types:
  - Categorical: ['gender', 'intent', 'regimen', 'disease_site', 'target_ED_visit', 'sodium', 'esas_pain', 'cycle_number', 'esas_tiredness', 'esas_depression', 'eosinophil', 'monocyte', 'phosphate', 'esas_drowsiness', 'esas_well_being', 'age', 'bicarbonate', 'magnesium', 'mean_platelet_volume', 'num_prior_ED_visits_within_5_years', 'chloride', 'esas_anxiety', 'hemoglobin', 'albumin', 'esas_shortness_of_breath', 'height', 'esas_appetite', 'total_bilirubin', 'potassium', 'creatinine', 'patient_ecog', 'mean_corpuscular_hemoglobin_concentration', 'esas_nausea']
  - Continuous: ['red_cell_distribution_width', 'body_surface_area', 'mean_corpuscular_hemoglobin', 'aspartate_aminotransferase', 'red_blood_cell', 'lymphocyte', 'neutrophil', 'white_blood_cell', 'mean_corpuscular_volume', 'alanine_aminotransferase', 'days_since_prev_ED_visit', 'platelet', 'weight'

In [4]:
import yaml
from pathlib import Path

config['columns']['categorical'] = ['gender', 'target_ED_visit', 'intent', 'regimen', 'disease_site']
config['columns']['continuous'] = [
    "glucose",
    "neutrophil",
    "aspartate_aminotransferase",
    "hematocrit",
    "weight",
    "white_blood_cell",
    "red_blood_cell",
    "mean_corpuscular_volume",
    "body_surface_area",
    "red_cell_distribution_width",
    "platelet",
    "mean_corpuscular_hemoglobin",
    "alanine_aminotransferase",
    "days_since_prev_ED_visit",
    "lactate_dehydrogenase",
    "lymphocyte",
    "alkaline_phosphatase",
    "esas_nausea",
    "age",
    "hemoglobin",
    "patient_ecog",
    "creatinine",
    "chloride",
    "phosphate",
    "esas_anxiety",
    "sodium",
    "potassium",
    "esas_depression",
    "total_bilirubin",
    "esas_well_being",
    "bicarbonate",
    "eosinophil",
    "height",
    "mean_corpuscular_hemoglobin_concentration",
    "esas_pain",
    "num_prior_ED_visits_within_5_years",
    "albumin",
    "esas_drowsiness",
    "monocyte",
    "cycle_number",
    "esas_shortness_of_breath",
    "esas_tiredness",
    "mean_platelet_volume",
    "esas_appetite",
    "magnesium"
]

output_dir = Path('a2r_outputs/analyzer/')
output_dir.mkdir(parents=True, exist_ok=True)

with open(output_dir / 'config.yaml', 'w') as f:
            yaml.dump(config, f)

In [5]:
from AutoML.analyzer import Analyzer

analyzer = Analyzer(df, target_variable='target_ED_visit', output_dir='./a2r_outputs/analyzer', config='./a2r_outputs/analyzer/config.yaml')

analyzer.run()

Feature Types:
  - Categorical: ['gender', 'target_ED_visit', 'intent', 'regimen', 'disease_site']
  - Continuous: ['glucose', 'neutrophil', 'aspartate_aminotransferase', 'hematocrit', 'weight', 'white_blood_cell', 'red_blood_cell', 'mean_corpuscular_volume', 'body_surface_area', 'red_cell_distribution_width', 'platelet', 'mean_corpuscular_hemoglobin', 'alanine_aminotransferase', 'days_since_prev_ED_visit', 'lactate_dehydrogenase', 'lymphocyte', 'alkaline_phosphatase', 'esas_nausea', 'age', 'hemoglobin', 'patient_ecog', 'creatinine', 'chloride', 'phosphate', 'esas_anxiety', 'sodium', 'potassium', 'esas_depression', 'total_bilirubin', 'esas_well_being', 'bicarbonate', 'eosinophil', 'height', 'mean_corpuscular_hemoglobin_concentration', 'esas_pain', 'num_prior_ED_visits_within_5_years', 'albumin', 'esas_drowsiness', 'monocyte', 'cycle_number', 'esas_shortness_of_breath', 'esas_tiredness', 'mean_platelet_volume', 'esas_appetite', 'magnesium']


Outlier Analysis:
  - No Outliers found in g

In [6]:
from AutoML.trainer import TrainerSupervised

df = pd.read_csv('./a2r_outputs/analyzer/updated_data.csv', index_col=0)

trainer = TrainerSupervised(task='binary', output_dir='./a2r_outputs/ED_trainer_explainer',)
trainer.run(df, 'target_ED_visit', k_folds=5)

Training fold 1/5...
Fold 1 score: 0.8879105679073933
Training fold 2/5...
Fold 2 score: 0.8855627123320744
Training fold 3/5...
Fold 3 score: 0.9034914564184813
Training fold 4/5...
Fold 4 score: 0.8921421625754349
Training fold 5/5...
Fold 5 score: 0.8745095717387014

Model Leaderboard (Displays values in "mean [min, max]" format across training folds)
------------------------------------------------------------------------------------
╒═══════════════════════╤══════════════════════════╤══════════════════════════╤══════════════════════════╕
│ model                 │ score_test               │ score_val                │ score_train              │
╞═══════════════════════╪══════════════════════════╪══════════════════════════╪══════════════════════════╡
│ WeightedEnsemble_L2   │ AUROC 0.87 [0.87, 0.87]  │ AUROC 0.89 [0.87, 0.9]   │ AUROC 1.0 [1.0, 1.0]     │
│                       │ F1: 0.24 [0.14, 0.29]    │ F1: 0.2 [0.08, 0.27]     │ F1: 0.95 [0.77, 1.0]     │
│                      

In [7]:
from AutoML.explainer import Explainer

exp = Explainer.from_trainer(trainer)
exp.run()

Subgroup Analysis(Disease_Site)
╒══════════════════════════════╤═══════════════════════╤══════════════════════╤════════╤═══════════════════════╕
│                              │ C15                   │ C16                  │ C17    │ C18                   │
╞══════════════════════════════╪═══════════════════════╪══════════════════════╪════════╪═══════════════════════╡
│ mean_prediction              │ 0.0457516339869281    │ 0.037383177570093455 │ 0.0    │ 0.027243589743589744  │
├──────────────────────────────┼───────────────────────┼──────────────────────┼────────┼───────────────────────┤
│ false_positive_rate          │ 0.015503875968992248  │ 0.006779661016949152 │ 0.0    │ 0.012704174228675136  │
├──────────────────────────────┼───────────────────────┼──────────────────────┼────────┼───────────────────────┤
│ Relative mean_prediction     │ 3.8306595365418894 ✅ │ 3.129991503823279 ✅ │ 0.0 ✅ │ 2.2810314685314683 ✅ │
├──────────────────────────────┼───────────────────────┼────────────

  0%|          | 0/100 [00:00<?, ?it/s]